# Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import numpy as np
from scipy.stats.mstats import mode
import seaborn as sns
import os
from glob import glob
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline 

sns.set_style('darkgrid')
sns.set_context('notebook')

In [2]:
csv_files = glob("./q2Data/*.csv.gz")
csv_files

['./q2Data\\imdb.name.basics.csv.gz',
 './q2Data\\imdb.title.akas.csv.gz',
 './q2Data\\imdb.title.basics.csv.gz',
 './q2Data\\imdb.title.crew.csv.gz',
 './q2Data\\imdb.title.principals.csv.gz',
 './q2Data\\imdb.title.ratings.csv.gz']

In [3]:
type(csv_files)

list

In [4]:
d = dict()
for file in csv_files:
    d[file] = pd.read_csv(file)

In [5]:
csv_files_dict = dict()
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv","").replace(".gz","").replace(".","_") # cleaning file names
    filename_df = pd.read_csv(filename, index_col=None)
    csv_files_dict[filename_cleaned] = filename_df

In [6]:
csv_files_dict.keys()

dict_keys(['imdb_name_basics', 'imdb_title_akas', 'imdb_title_basics', 'imdb_title_crew', 'imdb_title_principals', 'imdb_title_ratings'])

In [7]:
conn = sqlite3.Connection('./q2Data/movies_db.sqlite')

In [8]:
def create_sql_table_from_df(df, name, conn):
    try:
        df.to_sql(name, conn)
        print(f"Created sql table {name}")
    
    except Exception as e:
        print(f"Couldn't make sql table {name}")
        print(e)

In [9]:
for name, table in csv_files_dict.items():
    create_sql_table_from_df(table, name, conn)

Couldn't make sql table imdb_name_basics
Table 'imdb_name_basics' already exists.
Couldn't make sql table imdb_title_akas
Table 'imdb_title_akas' already exists.
Couldn't make sql table imdb_title_basics
Table 'imdb_title_basics' already exists.
Couldn't make sql table imdb_title_crew
Table 'imdb_title_crew' already exists.
Couldn't make sql table imdb_title_principals
Table 'imdb_title_principals' already exists.
Couldn't make sql table imdb_title_ratings
Table 'imdb_title_ratings' already exists.


In [10]:
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('imdb_name_basics',),
 ('imdb_title_akas',),
 ('imdb_title_basics',),
 ('imdb_title_crew',),
 ('imdb_title_principals',),
 ('imdb_title_ratings',)]

In [11]:
for item in csv_files_dict.items():
    print("******** NEW DF **********")
    print(item[0])
    print(item[1].info())
    print("******** END DF **********")

******** NEW DF **********
imdb_name_basics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            82736 non-null float64
death_year            6783 non-null float64
primary_profession    555308 non-null object
known_for_titles      576444 non-null object
dtypes: float64(2), object(4)
memory usage: 27.8+ MB
None
******** END DF **********
******** NEW DF **********
imdb_title_akas
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
title_id             331703 non-null object
ordering             331703 non-null int64
title                331703 non-null object
region               278410 non-null object
language             41715 non-null object
types                168447 non-null object
attributes           14925 non-null object
is_original_title    331678 non-nu

# 1 Data Cleaning

## 1.1 Dealing with datatypes

In [12]:
### change dtype of columns
# we want to change the types of 3 columns. Good idea to write a function for this

def convert_amt_to_int(df, col):
    df[col] = df[col].str.replace("$","").str.replace(",","").astype('float64')
    return df

## 1.2 Dealing with missing values
### 1.2.1 Removing empty columns from all dataframes

In [13]:
### scan through all columns and check portion of missing data
# we want to delete columns with more than 80% of data missing

def remove_empty_cols(df, col, cut):
    if df[col].isna().value_counts(normalize=True, sort=False)[0] < cut: # False < 20% (or True > 80%)
        print(f"\t col['{col}'] deleted b/c {round(100-df[col].isna().value_counts(normalize=True,sort=False)[0]*100,2)}% data missing.")
        df.drop(col, axis=1, inplace=True)
    else:
        print(f"\t col['{col}'] stayed.")
    return df

In [14]:
for nm, tb in csv_files_dict.items():
    print(f"\n")
    print(f"Viewing dataframe: {nm}")
    for tc in tb.columns:
        remove_empty_cols(tb, tc, 0.2) # if only less than 20% data, column removed. otherwise, kept! 



Viewing dataframe: imdb_name_basics
	 col['nconst'] stayed.
	 col['primary_name'] stayed.
	 col['birth_year'] deleted b/c 86.36% data missing.
	 col['death_year'] deleted b/c 98.88% data missing.
	 col['primary_profession'] stayed.
	 col['known_for_titles'] stayed.


Viewing dataframe: imdb_title_akas
	 col['title_id'] stayed.
	 col['ordering'] stayed.
	 col['title'] stayed.
	 col['region'] stayed.
	 col['language'] deleted b/c 87.42% data missing.
	 col['types'] stayed.
	 col['attributes'] deleted b/c 95.5% data missing.
	 col['is_original_title'] stayed.


Viewing dataframe: imdb_title_basics
	 col['tconst'] stayed.
	 col['primary_title'] stayed.
	 col['original_title'] stayed.
	 col['start_year'] stayed.
	 col['runtime_minutes'] stayed.
	 col['genres'] stayed.


Viewing dataframe: imdb_title_crew
	 col['tconst'] stayed.
	 col['directors'] stayed.
	 col['writers'] stayed.


Viewing dataframe: imdb_title_principals
	 col['tconst'] stayed.
	 col['ordering'] stayed.
	 col['nconst'] st

### 1.2.2 Deleting one individual column (dim. reduction) completely from one dataframe

In [15]:
def remove_column(df, col):
    df.drop(col, axis=1, inplace=True)
    return df

### 1.2.3 Deleting one individual row (redundant removal) completely from one dataframe

In [16]:
def remove_row(df, rowidx):
    df.drop(rowidx, axis=0, inplace=True)
    return df

### 1.2.4 Deleting several empty rows from one given column in one dataframe

In [17]:
def remove_empty_rows(df, col, cut): # if less than 10% data missing in one given column, deleted. otherwise, kept!
    rm_rows = list(df[df[col].isna()].index)
    print(f"empty row portion {(len(rm_rows)/df.shape[0])*100}% in col['{col}']")
    if len(rm_rows)/df.shape[0] < cut:
        df.dropna(subset=[col], how='any', thresh=1, inplace=True)
    return df

### 1.2.5 Checking and Removing duplicated rows from all dataframes

In [18]:
# we require ALL duplicated rows be deleted

def check_duplicate_rows(df):
    df['is_duplicated'] = df.duplicated(subset=None, keep='first')
    if df.is_duplicated.value_counts(normalize=True, sort=False)[0] == 1: # 100% False 
        print(f'\t no duplicated rows found!')
        df.drop('is_duplicated', axis=1, inplace=True)
        return False
    else:
        print(f'\t duplicated rows found!')
        df.drop('is_duplicated', axis=1, inplace=True)
        return True

In [19]:
def remove_duplicate_rows(name, df):
    try:
        df.drop_duplicates()
        print(f"Dropped duplicated rows in dataframe {name}")
        
    except Exception as e:
        print(f"Couldn't find duplicated rows in dataframe {name}")
        print(e)

In [20]:
for nm, tb in csv_files_dict.items():
    print(f"Viewing dataframe: {nm}")
    if check_duplicate_rows(tb):
        remove_duplicate_rows(nm, tb)

Viewing dataframe: imdb_name_basics
	 no duplicated rows found!
Viewing dataframe: imdb_title_akas
	 no duplicated rows found!
Viewing dataframe: imdb_title_basics
	 no duplicated rows found!
Viewing dataframe: imdb_title_crew
	 no duplicated rows found!
Viewing dataframe: imdb_title_principals
	 no duplicated rows found!
Viewing dataframe: imdb_title_ratings
	 no duplicated rows found!


### 1.2.6 Sorting one dataframe by one given column in descending way

In [21]:
def sorted_dataframe(df, col):
    df.sort_values(by=col, ascending=False, inplace=True)
    return df

## 1.3 Combining dataframes
### 1.3.1 Pandas Join/Merge

we want to combine five tables together. before doing that, lets check their dimensions.

In [22]:
print(csv_files_dict['imdb_title_ratings'].shape)
print(csv_files_dict['imdb_title_crew'].shape)
print(csv_files_dict['imdb_title_basics'].shape)
print(csv_files_dict['imdb_title_principals'].shape)
print(csv_files_dict['imdb_name_basics'].shape)

(73856, 3)
(146144, 3)
(146144, 6)
(1028186, 5)
(606648, 4)


In [23]:
names_df = pd.merge(csv_files_dict['imdb_name_basics'], csv_files_dict['imdb_title_principals'], on='nconst', how='inner')
print(names_df.shape)
names_df.head(20)

(1027912, 8)


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9,producer,NaN
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt0433397,7,composer,NaN
2,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt1681372,8,composer,NaN
3,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2387710,8,composer,NaN
4,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2281215,7,composer,NaN
5,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2749258,8,composer,NaN
6,nm0061865,Joseph Bauer,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt3564200,9,composer,NaN
7,nm0062070,Bruce Baum,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898",tt6463956,4,actor,"[""Shecky""]"
8,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387",tt2547632,10,cinematographer,NaN
9,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387",tt7697870,8,cinematographer,NaN


In [24]:
genres_df = pd.merge(names_df, csv_files_dict['imdb_title_basics'], on='tconst', how='inner')
print(genres_df.shape)
genres_df.head(20)

(1027912, 13)


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
2,nm0449549,Jordan Kerner,"producer,actor,executive","tt0101921,tt0141369,tt0119190,tt0108333",tt2398241,10,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
3,nm0962596,Pamela Ribon,"writer,producer,actress","tt0898332,tt3521164,tt5848272,tt2458776",tt2398241,7,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
4,nm0678963,Peyo,"writer,director,music_department","tt2017020,tt0081933,tt0074539,tt0472181",tt2398241,8,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
5,nm0542133,Joe Manganiello,"actor,stunts,producer","tt0974015,tt2268016,tt0844441,tt0837156",tt2398241,3,actor,"[""Hefty Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
6,nm0933988,Rainn Wilson,"actor,producer,soundtrack","tt1512235,tt0467406,tt0386676,tt0465624",tt2398241,2,actor,"[""Gargamel""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
7,nm1416215,Demi Lovato,"soundtrack,actress,music_department","tt0413573,tt1196339,tt1055366,tt1252374",tt2398241,1,actress,"[""Smurfette""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
8,nm1632630,Stacey Harman,"writer,miscellaneous,producer","tt2398241,tt7221388,tt5662574,tt2712740",tt2398241,6,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"
9,nm1442113,Jack McBrayer,"actor,soundtrack,producer","tt0800039,tt0496424,tt0415306,tt1772341",tt2398241,4,actor,"[""Clumsy Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy"


In [25]:
directors_df = pd.merge(genres_df, csv_files_dict['imdb_title_crew'], on='tconst', how='inner')
print(directors_df.shape)
directors_df.head(20)

(1027912, 15)


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
2,nm0449549,Jordan Kerner,"producer,actor,executive","tt0101921,tt0141369,tt0119190,tt0108333",tt2398241,10,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
3,nm0962596,Pamela Ribon,"writer,producer,actress","tt0898332,tt3521164,tt5848272,tt2458776",tt2398241,7,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
4,nm0678963,Peyo,"writer,director,music_department","tt2017020,tt0081933,tt0074539,tt0472181",tt2398241,8,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
5,nm0542133,Joe Manganiello,"actor,stunts,producer","tt0974015,tt2268016,tt0844441,tt0837156",tt2398241,3,actor,"[""Hefty Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
6,nm0933988,Rainn Wilson,"actor,producer,soundtrack","tt1512235,tt0467406,tt0386676,tt0465624",tt2398241,2,actor,"[""Gargamel""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
7,nm1416215,Demi Lovato,"soundtrack,actress,music_department","tt0413573,tt1196339,tt1055366,tt1252374",tt2398241,1,actress,"[""Smurfette""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
8,nm1632630,Stacey Harman,"writer,miscellaneous,producer","tt2398241,tt7221388,tt5662574,tt2712740",tt2398241,6,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"
9,nm1442113,Jack McBrayer,"actor,soundtrack,producer","tt0800039,tt0496424,tt0415306,tt1772341",tt2398241,4,actor,"[""Clumsy Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963"


In [26]:
ratings_df = pd.merge(directors_df, csv_files_dict['imdb_title_ratings'], on='tconst', how='inner')
print(ratings_df.shape)
ratings_df.head(20)

(629598, 17)


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
2,nm0449549,Jordan Kerner,"producer,actor,executive","tt0101921,tt0141369,tt0119190,tt0108333",tt2398241,10,producer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
3,nm0962596,Pamela Ribon,"writer,producer,actress","tt0898332,tt3521164,tt5848272,tt2458776",tt2398241,7,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
4,nm0678963,Peyo,"writer,director,music_department","tt2017020,tt0081933,tt0074539,tt0472181",tt2398241,8,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
5,nm0542133,Joe Manganiello,"actor,stunts,producer","tt0974015,tt2268016,tt0844441,tt0837156",tt2398241,3,actor,"[""Hefty Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
6,nm0933988,Rainn Wilson,"actor,producer,soundtrack","tt1512235,tt0467406,tt0386676,tt0465624",tt2398241,2,actor,"[""Gargamel""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
7,nm1416215,Demi Lovato,"soundtrack,actress,music_department","tt0413573,tt1196339,tt1055366,tt1252374",tt2398241,1,actress,"[""Smurfette""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
8,nm1632630,Stacey Harman,"writer,miscellaneous,producer","tt2398241,tt7221388,tt5662574,tt2712740",tt2398241,6,writer,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
9,nm1442113,Jack McBrayer,"actor,soundtrack,producer","tt0800039,tt0496424,tt0415306,tt1772341",tt2398241,4,actor,"[""Clumsy Smurf""]",Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612


In [27]:
directors_only_df = ratings_df[ratings_df.ordering==5]
print(directors_only_df.shape)
directors_only_df.head(20)

(67622, 17)


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
11,nm0647487,Christopher Olness,"writer,producer,director","tt0160452,tt0117060,tt0433397,tt2410746",tt0433397,5,director,NaN,Satin,Satin,2011,84.0,"Comedy,Drama,Music",nm0647487,"nm0647487,nm0903200",4.7,214
27,nm1490835,Joe Randazzo,"producer,actor,writer","tt4074034,tt0877343,tt1408412,tt1681372",tt1681372,5,writer,NaN,Caesar and Otto's Deadly Xmas,Caesar and Otto's Deadly Xmas,2012,83.0,"Comedy,Horror",nm0996586,"nm0996586,nm1490835",4.0,127
32,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2387710,5,director,NaN,Journey to the Forbidden Valley,Journey to the Forbidden Valley,2017,96.0,"Action,Adventure,Family",nm0928832,"nm0906901,nm0928832",5.4,50
42,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2281215,5,director,NaN,Elf-Man,Elf-Man,2012,86.0,"Comedy,Family,Fantasy",nm0928832,"nm0928832,nm0420135",3.6,582
53,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt2749258,5,director,NaN,Somebody Marry Me,Somebody Marry Me,2013,101.0,"Comedy,Romance",nm0038875,nm0038875,4.9,56
67,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt3564200,5,director,NaN,Tooken,Tooken,2015,80.0,"Action,Comedy",nm0038875,"nm0887226,nm0038875,nm8357383",3.6,1388
72,nm0052045,Mirra Bank,"editor,director,producer","tt0253746,tt0316160,tt0074605,tt0085494",tt2547632,5,director,NaN,The Only Real Game,The Only Real Game,2013,82.0,"Documentary,Sport",nm0052045,NaN,7.3,23
82,nm2383548,Joshua Bennett,"producer,director,production_manager","tt7697866,tt7697868,tt2193091,tt7697870",tt7697870,5,director,NaN,Sky and Ground,Sky and Ground,2018,86.0,Documentary,"nm3865965,nm2383548",NaN,8.7,23
92,nm0509032,Max Avery Lichtenstein,"composer,soundtrack,music_department","tt0396688,tt0339638,tt0297884,tt0186253",tt8171864,5,composer,NaN,Grit,Grit,2018,80.0,Documentary,"nm1106629,nm4299959",NaN,7.6,14


In [28]:
# delete empty rows from 'directors' column

row_len = directors_only_df.shape[0]
remove_empty_rows(directors_only_df, 'directors', 1.0)
print(directors_only_df.shape)
print(directors_only_df.directors.isna().value_counts())
print(round(directors_only_df.shape[0]/row_len,3))
directors_only_df.head(20)

empty row portion 0.5072313744047795% in col['directors']
(67279, 17)
False    67279
Name: directors, dtype: int64
0.995


C:\Users\renjm\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
11,nm0647487,Christopher Olness,"writer,producer,director","tt0160452,tt0117060,tt0433397,tt2410746",tt0433397,5,director,NaN,Satin,Satin,2011,84.0,"Comedy,Drama,Music",nm0647487,"nm0647487,nm0903200",4.7,214
27,nm1490835,Joe Randazzo,"producer,actor,writer","tt4074034,tt0877343,tt1408412,tt1681372",tt1681372,5,writer,NaN,Caesar and Otto's Deadly Xmas,Caesar and Otto's Deadly Xmas,2012,83.0,"Comedy,Horror",nm0996586,"nm0996586,nm1490835",4.0,127
32,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2387710,5,director,NaN,Journey to the Forbidden Valley,Journey to the Forbidden Valley,2017,96.0,"Action,Adventure,Family",nm0928832,"nm0906901,nm0928832",5.4,50
42,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2281215,5,director,NaN,Elf-Man,Elf-Man,2012,86.0,"Comedy,Family,Fantasy",nm0928832,"nm0928832,nm0420135",3.6,582
53,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt2749258,5,director,NaN,Somebody Marry Me,Somebody Marry Me,2013,101.0,"Comedy,Romance",nm0038875,nm0038875,4.9,56
67,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt3564200,5,director,NaN,Tooken,Tooken,2015,80.0,"Action,Comedy",nm0038875,"nm0887226,nm0038875,nm8357383",3.6,1388
72,nm0052045,Mirra Bank,"editor,director,producer","tt0253746,tt0316160,tt0074605,tt0085494",tt2547632,5,director,NaN,The Only Real Game,The Only Real Game,2013,82.0,"Documentary,Sport",nm0052045,NaN,7.3,23
82,nm2383548,Joshua Bennett,"producer,director,production_manager","tt7697866,tt7697868,tt2193091,tt7697870",tt7697870,5,director,NaN,Sky and Ground,Sky and Ground,2018,86.0,Documentary,"nm3865965,nm2383548",NaN,8.7,23
92,nm0509032,Max Avery Lichtenstein,"composer,soundtrack,music_department","tt0396688,tt0339638,tt0297884,tt0186253",tt8171864,5,composer,NaN,Grit,Grit,2018,80.0,Documentary,"nm1106629,nm4299959",NaN,7.6,14


In [29]:
# delete empty rows from 'writers' column

row_len = directors_only_df.shape[0]
remove_empty_rows(directors_only_df, 'writers', 1.0)
print(directors_only_df.shape)
print(directors_only_df.writers.isna().value_counts())
print(round(directors_only_df.shape[0]/row_len,3))
directors_only_df.head(20)

empty row portion 10.964788418377205% in col['writers']
(59902, 17)
False    59902
Name: writers, dtype: int64
0.89


C:\Users\renjm\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
11,nm0647487,Christopher Olness,"writer,producer,director","tt0160452,tt0117060,tt0433397,tt2410746",tt0433397,5,director,NaN,Satin,Satin,2011,84.0,"Comedy,Drama,Music",nm0647487,"nm0647487,nm0903200",4.7,214
27,nm1490835,Joe Randazzo,"producer,actor,writer","tt4074034,tt0877343,tt1408412,tt1681372",tt1681372,5,writer,NaN,Caesar and Otto's Deadly Xmas,Caesar and Otto's Deadly Xmas,2012,83.0,"Comedy,Horror",nm0996586,"nm0996586,nm1490835",4.0,127
32,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2387710,5,director,NaN,Journey to the Forbidden Valley,Journey to the Forbidden Valley,2017,96.0,"Action,Adventure,Family",nm0928832,"nm0906901,nm0928832",5.4,50
42,nm0928832,Ethan Wiley,"writer,director,producer","tt2281215,tt0093220,tt5794440,tt0150111",tt2281215,5,director,NaN,Elf-Man,Elf-Man,2012,86.0,"Comedy,Family,Fantasy",nm0928832,"nm0928832,nm0420135",3.6,582
53,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt2749258,5,director,NaN,Somebody Marry Me,Somebody Marry Me,2013,101.0,"Comedy,Romance",nm0038875,nm0038875,4.9,56
67,nm0038875,John Asher,"actor,director,producer","tt3564200,tt2749258,tt1401621,tt0368530",tt3564200,5,director,NaN,Tooken,Tooken,2015,80.0,"Action,Comedy",nm0038875,"nm0887226,nm0038875,nm8357383",3.6,1388
104,nm1203596,Kim Mordaunt,"director,writer,actor","tt5390418,tt0326196,tt1444679,tt2178256",tt2178256,5,director,NaN,The Rocket,The Rocket,2013,96.0,Drama,nm1203596,nm1203596,7.3,2794
110,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt2057445,5,director,NaN,No Other Woman,No Other Woman,2011,101.0,"Drama,Romance,Thriller",nm0062879,"nm1282374,nm2874544,nm2752054",6.4,256
120,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt1592569,5,director,NaN,Paano na kaya,Paano na kaya,2010,110.0,"Drama,Romance",nm0062879,"nm1288308,nm2752054,nm4037950,nm4059063",6.4,77


In [30]:
row_len = directors_only_df.shape[0]
for idx in directors_only_df.index: 
    if directors_only_df.loc[idx, 'directors'] not in directors_only_df.loc[idx, 'writers']:
        if idx%1000 == 0:
            print(f"writers in row {idx} not same as directors. kept!")
    else:
        if idx%1000 == 0:
            print(f"writers in row {idx} also directors. removed!")
        directors_writers_sep_df = remove_row(directors_only_df, idx)

print(directors_writers_sep_df.shape)
print(round(directors_writers_sep_df.shape[0]/directors_only_df.shape[0],3))
directors_writers_sep_df.head(20)

C:\Users\renjm\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


writers in row 8000 not same as directors. kept!
writers in row 15000 also directors. removed!
writers in row 26000 also directors. removed!
writers in row 32000 also directors. removed!
writers in row 33000 also directors. removed!
writers in row 56000 also directors. removed!
writers in row 60000 not same as directors. kept!
writers in row 74000 also directors. removed!
writers in row 80000 also directors. removed!
writers in row 90000 also directors. removed!
writers in row 92000 also directors. removed!
writers in row 130000 also directors. removed!
writers in row 133000 also directors. removed!
writers in row 147000 also directors. removed!
writers in row 182000 also directors. removed!
writers in row 191000 also directors. removed!
writers in row 202000 not same as directors. kept!
writers in row 215000 not same as directors. kept!
writers in row 221000 also directors. removed!
writers in row 236000 also directors. removed!
writers in row 237000 not same as directors. kept!
write

,nconst,primary_name,primary_profession,known_for_titles,tconst,ordering,category,characters,primary_title,original_title,start_year,runtime_minutes,genres,directors,writers,averagerating,numvotes
1,nm0038432,Kelly Asbury,"art_department,animation_department,director","tt0298148,tt0101414,tt0166813,tt0377981",tt2398241,5,director,NaN,Smurfs: The Lost Village,Smurfs: The Lost Village,2017,90.0,"Adventure,Animation,Comedy",nm0038432,"nm1632630,nm0962596,nm0678963",6.0,15612
110,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt2057445,5,director,NaN,No Other Woman,No Other Woman,2011,101.0,"Drama,Romance,Thriller",nm0062879,"nm1282374,nm2874544,nm2752054",6.4,256
120,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt1592569,5,director,NaN,Paano na kaya,Paano na kaya,2010,110.0,"Drama,Romance",nm0062879,"nm1288308,nm2752054,nm4037950,nm4059063",6.4,77
130,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt2590280,5,director,NaN,One More Try,One More Try,2012,100.0,Drama,nm0062879,nm0911075,5.8,158
140,nm0062879,Ruel S. Bayani,"director,production_manager,miscellaneous","tt2590280,tt0352080,tt0216559,tt2057445",tt8421806,5,director,NaN,Kasal,Kasal,2018,115.0,NaN,nm0062879,"nm2752054,nm6747643",7.9,54
171,nm0400170,Ralf Huettner,"director,writer,actor","tt0097126,tt0112203,tt0130887,tt0264917",tt1611211,5,director,NaN,Vincent Wants to Sea,Vincent will Meer,2010,96.0,Drama,nm0400170,nm1164552,7.1,4955
215,nm1235530,Michael Urie,"actor,producer,director","tt2852262,tt1657299,tt2076216,tt0805669",tt2076216,5,director,NaN,He's Way More Famous Than You,He's Way More Famous Than You,2013,96.0,Comedy,nm1235530,"nm0270546,nm0816648",3.7,649
232,nm0200403,Michelle Danner,"director,producer,actress","tt0259280,tt3671052,tt1876330,tt0270417",tt1876330,5,director,NaN,Hello Herman,Hello Herman,2012,90.0,Drama,nm0200403,nm0537545,5.8,1236
244,nm0000770,David Anspaugh,"producer,director","tt0091217,tt0081873,tt0083483,tt0108002",tt1614430,5,director,NaN,Little Red Wagon,Little Red Wagon,2012,104.0,Drama,nm0000770,nm0242059,6.4,503
257,nm2360222,Pieter Gaspersz,"actor,producer,director","tt4487150,tt3052482,tt7070600,tt1365499",tt1147681,5,director,NaN,After,After,2014,99.0,"Drama,Mystery",nm2360222,nm0312905,4.7,129


In [ ]:
# making a list of all the cols where we want to delete

useless_cols = ['characters', 'runtime_minutes', 'writers']

for col in useless_cols:
    q2_ROI_df = remove_column(directors_only_df, col)

# 2 
## Question 2: What genres of 5 movies is each director best known for?

# 3
## Question 3:  Is most popular genre correlated with director's profession?